# Современные библиотеки для AI-агентов

### 📌 **LangChain**
---
[Библиотека](https://www.langchain.com/)


### **LangChain** – это фреймворк для разработки приложений на основе больших языковых моделей (LLM). 

LangChain ориентирован на гибкость и широкую совместимость: поддерживаются десятки провайдеров LLM **(OpenAI, Google Gemini, AWS, Anthropic и др.)** и множество инструментов **(поиск в Интернете, базы знаний, собственные API)**. 

Библиотека нацелена на быстрое прототипирование и производство приложений – в частности, для чат-ботов, систем вопрос-ответ с помощью Retrieval-Augmented Generation (RAG), автоматизации задач и других сценариев.


### Реализация основных компонентов LangChain

* **Цепочки (Chains)**: 

    - Базовый класс Chain наследуется от [RunnableSerializable](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.RunnableSerializable.html) и определяет параметры запуска (входы, конфигурацию, метод invoke). Цепочка управляет потоком данных: собирает входные переменные, передаёт их через последовательность вызовов (промпт -> модель -> парсер).

* **Шаблоны подсказок ([PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html))**: 
    - Класс PromptTemplate содержит строковый шаблон с переменными. Поддерживаются форматы f-string. При вызове он подставляет значения переменных (метод format / invoke). Можно создавать динамические шаблоны – например, частично заполнять их заранее (partial_variables), оставляя не все переменные для пользователя. 


* **Агенты (Agents)**: 
    - Класс AgentExecutor (наследник Chain) организует цикл работы агента. Он содержит agent (объект типа [BaseSingleActionAgent](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.BaseSingleActionAgent.html) или [RunnableAgent](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.RunnableAgent.html)), список Tools и логику остановки (максимум итераций, таймаут, метод досрочной остановки). 
    
    - При запуске [AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html) вызывает метод plan агента (обычно LLMChain или RunnableAgent) для принятия решения (какой инструмент вызвать или завершить работу), затем выполняет выбранный инструмент и сохраняет шаг как AgentAction. Так продолжается, пока агент не вернёт AgentFinish или не сработают критерии досрочной остановки.

* **Инструменты (Tools)**: 
    - Определены абстрактным классом [BaseTool](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.base.BaseTool.html). Инструмент
    имеет атрибуты name, description и опциональную схему аргументов (args_schema) – Pydantic-модель или JSON-схему длявалидации входов. Инструменты добавляются к агентам как доступные функции, агент может вызывать их по имени. Метаданные return_direct=True указывают, что после выполнения инструмента агент прекращает цикл. Toolkit – коллекция связанных инструментов. 


 * **Память (Memory)**: 
    - Интерфейс [BaseMemory](https://api.python.langchain.com/en/latest/memory/langchain_core.memory.BaseMemory.html) определяет хранилище состояния между вызовами цепочки. Классы наследники (например, буфер переписки) сохраняют прошлые сообщения или контекст, автоматически добавляя их в новые подсказки. Несмотря на устаревание старой
    модели памяти, идея такова: память хранит данные предыдущих шагов цепочки и инжектит их как переменные в входные данные следующего вызова. 

* **Шаблоны для чата ([ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html))**:
    - Расширяет PromptTemplate для диалоговых моделей. Принимает список сообщений с ролями (system, human, ai) и шаблонами текста.
    Например, ChatPromptTemplate([("system","{persona}"),("human","{question}")]) строит цепочку сообщений. 

In [ ]:
# pip install langchain langchain-community

In [1]:
MODEL_NAME = "mistral-nemo:12b-instruct-2407-q3_K_L"

### 1. LLM обертки
LangChain предоставляет обертки над LLM API, включая OpenAI, Anthropic, HuggingFaceHub и локальные модели (через llamacpp, Ollama, и др.).

In [4]:
from langchain_community.llms import Ollama

llm = Ollama(model=MODEL_NAME, temperature=0.7)
response = llm("Когда изобрели парацетамол?")
print(response)

Парацетамол (acetaminophen) был впервые синтезирован в 1879 году немецким химиком Йозефом фон Мёрингом, но его анальгетическое и антипиретические свойства не были обнаружены до 1893 года.ferred.


### 2. Prompt Templates
PromptTemplate помогает структурировать промпты и вставлять переменные.

In [5]:
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("Назови {n} столиц мира.")
prompt = template.format(n=3)
print(prompt)

Назови 3 столиц мира.


In [6]:
llm(prompt)

'Лондон, Париж, Вашингтон'

### 3. Chains (цепочки)
Chain — это композиция из шагов, включающих промпты, модели и пост обработку.

In [14]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=template)
output = chain.run(n=3)

In [15]:
print("Промпт:", template.template)
print("Ответ модели:", output)

Промпт: Назови {n} столиц мира.
Ответ модели: 1. Москва - столица России
2. Нью-Йорк - один из крупнейших городов мира, но официальная столица США находится в Вашингтоне.
3. Пекин - столица Китая


### 4. Agents

Агенты используют LLM для динамического выбора инструментов и выполнения задач.


In [21]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [24]:
from langchain.agents import initialize_agent, AgentType

tools = [wiki]
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent.run("Кто такой Петр I и чем он знаменит? Ответ нужен на русском языке.")




> Entering new AgentExecutor chain...
I need to find out who Peter I is and what he is famous for. I'll use Wikipedia in Russian.

Action: wikipedia
Action Input: Петр I
Observation: Page: Peter the Great
Summary: Peter I (Russian: Пётр I Алексеевич, romanized: Pyotr I Alekseyevich, IPA: [ˈpʲɵtr ɐlʲɪkˈsʲejɪvʲɪtɕ]; 9 June [O.S. 30 May] 1672
– 8 February [O.S. 28 January] 1725), better known as Peter the Great, was the Tsar of all Russia from 1682 and the first Emperor of all Russia from 1721 until his death in 1725. He reigned jointly with his half-brother Ivan V until 1696. From this year, Peter was an absolute monarch, an autocrat who remained the ultimate authority and organized a well-ordered police state.
Much of Peter's reign was consumed by lengthy wars against the Ottoman and Swedish empires. His Azov campaigns were followed by the foundation of the Russian Navy; after his victory in the Great Northern War, Russia annexed a significant portion of the eastern Baltic coastline a

'Пётр I, также известный как Пётр Великий, был Царем и позднее Императором России. Он прославился тем, что модернизировал Россию, превратив её в одну из ведущих европейских держав, и основал город Санкт-Петербург.'

In [ ]:
# Агент с БД

import sqlite3
from langchain.llms import Ollama
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

llm = Ollama(model="mistral-nemo:12b-instruct-2407-q3_K_L", temperature=0.4)

# создадим таблицу в БД для демонстрации
conn = sqlite3.connect("students.db")
cursor = conn.cursor()

cursor.executescript("""
DROP TABLE IF EXISTS students;
DROP TABLE IF EXISTS grades;

CREATE TABLE students (
    id INTEGER PRIMARY KEY,
    name TEXT,
    group_name TEXT
);

CREATE TABLE grades (
    id INTEGER PRIMARY KEY,
    student_id INTEGER,
    subject TEXT,
    grade INTEGER,
    FOREIGN KEY(student_id) REFERENCES students(id)
);
""")

students = [
    (1, "Алексей Иванов", "Группа А"),
    (2, "Мария Смирнова", "Группа Б"),
    (3, "Игорь Кузнецов", "Группа А"),
]
grades = [
    (1, 1, "Математика", 5),
    (2, 1, "Физика", 4),
    (3, 2, "Математика", 3),
    (4, 2, "Физика", 5),
    (5, 3, "Математика", 4),
    (6, 3, "Физика", 4),
]
cursor.executemany("INSERT INTO students VALUES (?, ?, ?)", students)
cursor.executemany("INSERT INTO grades VALUES (?, ?, ?, ?)", grades)
conn.commit()


db = SQLDatabase.from_uri("sqlite:///students.db")
# цепочка для langchain, подлючаем нашу БД
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

# обращаемся к БД 
question = "Какие оценки получил Алексей Иванов по каждому предмету?"
response = db_chain.run(question)
print("\n📘 Ответ:\n", response)

/var/folders/j_/mhkb8_q935160j788zvw62ym5_k29d/T/ipykernel_20899/3539935137.py:8: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral-nemo:12b-instruct-2407-q3_K_L", temperature=0.4)
/var/folders/j_/mhkb8_q935160j788zvw62ym5_k29d/T/ipykernel_20899/3539935137.py:59: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = db_chain.run(question)




> Entering new SQLDatabaseChain chain...
Какие оценки получил Алексей Иванов по каждому предмету?
SQLQuery:SELECT "subject", "grade"
FROM "grades"
JOIN "students" ON "grades"."student_id" = "students"."id"
WHERE "students"."name" = 'Алексей Иванов'
LIMIT 5;
SQLResult: [('Математика', 5), ('Физика', 4)]
Answer:Алексей Иванов получил оценки:
- по Математике - 5
- по Физике - 4
> Finished chain.

📘 Ответ:
 Алексей Иванов получил оценки:
- по Математике - 5
- по Физике - 4


### 6. Memory (Память)

Память позволяет хранить историю диалога:

In [25]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain


# память (история диалога)
memory = ConversationBufferMemory()

# создаём разговорную цепочку (чат)
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True  # чтобы видеть внутренние шаги
)

response_1 = conversation.predict(input="Привет, меня зовут Диана.")
print("\nОтвет 1:", response_1)

response_2 = conversation.predict(input="Как меня зовут?")
print("\n Ответ 2:", response_2)

/var/folders/j_/mhkb8_q935160j788zvw62ym5_k29d/T/ipykernel_6706/1888662594.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/var/folders/j_/mhkb8_q935160j788zvw62ym5_k29d/T/ipykernel_6706/1888662594.py:9: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Привет, меня зовут Диана.
AI:

> Finished chain.

Ответ 1: Привет, Диана! Я — языковая модель. Какой у вас прекрасный русский язык! Чем я могу вам помочь сегодня?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Привет, меня зовут Диана.
AI: Привет, Диана! Я — языковая модель. Какой у вас прекрасный русский язык! Чем я могу вам помочь сегодня?
Human: Как меня зовут?
AI:

> Finished cha

### 📌 **LangGraph**
---
[Библиотека](https://www.langchain.com/langgraph)


**LangGraph строит граф вычислений поверх модели LangChain.**

Основные сущности: 
- **StateGraph/Graph** – граф, вершины которого связаны направленными ребрами
- **Node** – узел, выполняющий функцию (Runnable или Python-функция)
- **Edge/ConditionalEdge** – переходы между узлами (обычные или условные)
- **State** – общее состояние графа (словарь), который разделяют все узлы.

В StateGraph узлы получают на вход текущее состояние и возвращают частичное обновление (например, новые значения для некоторых ключей) . 

In [ ]:
# pip install langgraph langchain langchain-community

In [ ]:
from typing import TypedDict
from langchain.agents import Tool, AgentExecutor, create_openai_functions_agent
from langchain.agents.tools import tool
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END

# схема состояния 
class AgentState(TypedDict):
    input: str
    final_answer: str | None

@tool
def calculator(expression: str) -> str:
    """Вычисляет простое математическое выражение."""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Ошибка: {e}"

tools = [calculator]

llm = ChatOllama(model=MODEL_NAME)   

prompt = ChatPromptTemplate.from_messages([
    ("system", "Ты — умный помощник с доступом к инструментам."),
    ("user", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

# агент и исполнитель
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# ноды (узлы) графа
def agent_step(state: AgentState) -> AgentState:
    result = agent_executor.invoke({"input": state["input"]}) 
    return {**state, "final_answer": result["output"]}

def should_continue(state: AgentState) -> str:
    return "end" if state["final_answer"] else "agent"

# строим сам граф
builder = StateGraph(state_schema=AgentState)
builder.add_node("agent", RunnableLambda(agent_step))
builder.set_entry_point("agent")
builder.add_conditional_edges(
    "agent",
    RunnableLambda(should_continue),
    {"agent": "agent", "end": END}
)
graph = builder.compile()

init_state: AgentState = {"input": "Сколько будет (25 + 5) * 2?", "final_answer": None}
result = graph.invoke(init_state)

print("Финальный ответ:", result["final_answer"])



> Entering new AgentExecutor chain...
Сначала выполняем действие в скобках, а затем умножение:

(25 + 5) = 30
30 * 2 = 60

Ответ: 60.

> Finished chain.
Финальный ответ: Сначала выполняем действие в скобках, а затем умножение:

(25 + 5) = 30
30 * 2 = 60

Ответ: 60.


### 📌 **AutoGen**
---
[Библиотека](https://microsoft.github.io/autogen/stable//index.html)


**AutoGen** – это фреймворк от Microsoft Research для создания многопроцессных (multiagent) AI-приложений . Он предназначен для разработки агентных систем, где несколько LLMагентов могут общаться друг с другом или с людьми для решения задач. AutoGen реализует асинхронную событийно-ориентированную архитектуру: агенты обмениваются сообщениями в разных режимах (запрос/ответ, события). 

Благодаря этому можно строить детерминированные и динамические бизнес-процессы с поддержкой совместной работы агентов. 


**В отличие от LangChain, AutoGen предоставляет асинхронные, событийные и диалоговые LLM-сценарии, где агенты ведут переписку, передавая сообщения и делая паузы для обработки.**


<img src="./pics/assistant-agent.svg" width="75%">

##### Есть основной "разговорный" агент ConversableAgent и его подклассы:


- [AssistantAgent](https://microsoft.github.io/autogen/0.2/docs/reference/agentchat/assistant_agent/#assistantagent) – это ассистент ИИ, который использует LLM для генерации ответов и может вызывать инструменты и функции. 


- [UserProxyAgent](https://microsoft.github.io/autogen/0.2/docs/reference/agentchat/user_proxy_agent)– это агент-прокси для человека: по умолчанию запрашивает ввод пользователя, но при наличии блоков кода может выполнять их автоматически.


#### Агенты общаются посредством AgentRuntime – централизованного обмена сообщениями, реализованного по принципам акторной модели. 

AutoGen поддерживает два режима выполнения: 
- **StandaloneRuntime (однопроцессный)**

<img src="./pics/architecture-standalone.svg" width="35%">

- **Distributed Runtime (с несколькими хостами/воркерами)**

<img src="./pics/architecture-distributed.svg" width="35%">

### Независимо от режима, все агенты регистрируются в рантайме, после чего к ним можно посылать сообщения. Для адресной пересылки используются методы:

* **send_message(msg, recipient_id, ...)** – отправка с ожиданием ответа,

или

* **publish_message(msg, topic_id, ...)** – широковещательная публикация без ожидания ответа 

Таким образом, «протокол» взаимодействия – это асинхронный обмен объектами сообщений (текстами, объектами типа FunctionCall , ToolExecutionResult и пр.) через шину рантайма.

## Шаблоны промптов и системное сообщение в AutoGen
Каждый агент имеет конфигурацию системного сообщения (system prompt/message), которое задаётся при создании агента или берётся по умолчанию.

In [ ]:
# Внутри AssistantAgent: 

system_message: (
            str | None
        ) = """You are a helpful AI assistant. Solve tasks using your tools. 
            Reply with TERMINATE when the task has been completed."""

# Create an AssistantAgent instance that uses the tool and model client.
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[tool],
    system_message="Use the tool to analyze sentiment.",
    output_content_type=AgentResponse)

# Create an AssistantAgent instance with the model client and context.
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    model_context=model_context,
    system_message="You are a helpful assistant.",
)

In [30]:
# Внутри CodeExecutorAgent: 
# It is recommended that the `CodeExecutorAgent` agent uses a Docker container to execute code. 
# This ensures that model-generated code is executed in an isolated environment. 
# To use Docker, your environment must have Docker installed and running.

DEFAULT_SYSTEM_MESSAGE = """You are a Code Execution Agent. 
Your role is to generate and execute Python code and shell scripts based on user instructions, 
ensuring correctness, efficiency, and minimal errors. Handle edge cases gracefully.
Python code should be provided in ```python code blocks, and sh shell scripts should be provided in ```sh code blocks for execution."""


### Настройка окружения, роли и интеграция инструментов

AutoGen предоставляет богатые возможности по подключению инструментов и сред.

- Во-первых, память агентов реализована через классы Memory (например, ListMemory ) из autogen_core.memory. Память может накапливать произвольные записи и при необходимости подгружать их в контекст чата.

- Во-вторых, инструменты оформлены как наследники BaseTool:
    - FunctionTool для обертки произвольной функции Python (AssistantAgent)
    - ToolAgent (ядро для исполнения FunctionCall -сообщений в autogen_core.tool_agent)
    - Workbench – коллекция таких инструментов. 

AutoGen оптимизирован для исследовательских и производственных многозадачных сценариев: его архитектура позволяет запускать долгоживущих агентов с наблюдением и интеграцией в рабочие процессы.

Ключевые возможности и сценарии:
- **Асинхронные агенты**: обмен сообщениями без блокировки, можно строить сложные рабочие процессы с несколькими шагами.

- **Модульность**: легко добавлять свои модели, агенты, инструменты и память; поддержка «продвинутых» возможностей (агентов-утилит, командная работа агентов).

- **Наблюдаемость**: инструменты для трассировки взаимодействий агентов, аналитики, логирования, поддержка OpenTelemetry.

- **Распределённость**: можно запускать агентов на нескольких узлах или микросервисах.

- **Примеры применения**: автоматизация бизнес-процессов (определённая последовательность действий), исследовательские задачи по коллаборации агентов, распределённые агенты для разных языков и т.д. 

In [25]:
llm_config = {
    'model': 'mistral-nemo:12b-instruct-2407-q3_K_L',
    'base_url': 'http://localhost:11434/v1',
    'api_key': 'ollama',
    'cache_seed': None,
    'temperature': 0.2
}

In [ ]:
from autogen import AssistantAgent, UserProxyAgent


# cоздание агентов. Это всегда AssistantAgent
analyst = AssistantAgent(
    name="Аналитик",
    system_message="Ты профессиональный аналитик данных. Генерируй код на Python",
    llm_config=llm_config
)

# по сути это пользователь
user_proxy = UserProxyAgent(
    name="Пользователь",
    human_input_mode="NEVER", 
    code_execution_config={"work_dir": "coding", "use_docker": False},
    max_consecutive_auto_reply=3
)

# здесь наш диалог, где будет работать 
user_proxy.initiate_chat(
    analyst,
    message="""Задача:
    1. Сгенерируй 1000 рандомных чисел
    2. Рассчитай среднее и стандартное отклонение"""
)

Пользователь (to Аналитик):

Задача:
    1. Сгенерируй 1000 рандомных чисел
    2. Рассчитай среднее и стандартное отклонение

--------------------------------------------------------------------------------
[autogen.oai.client: 06-29 15:57:55] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Аналитик (to Пользователь):

Вот как вы можете сгенерировать 1000 случайных чисел, рассчитать их среднее и стандартное отклонение в Python:

```python
import numpy as np

# Шаг 1: Сгенерируйте 1000 рандомных чисел
random_numbers = np.random.rand(1000)

# Шаг 2: Рассчитайте среднее и стандартное отклонение
mean_value = np.mean(random_numbers)
std_deviation = np.std(random_numbers)

print("Среднее значение:", mean_value)
print("Стандартное отклонение:", std_deviation)
```

В этом коде мы используем библиотеку NumPy для генерации случайны

ChatResult(chat_id=None, chat_history=[{'content': 'Задача:\n    1. Сгенерируй 1000 рандомных чисел\n    2. Рассчитай среднее и стандартное отклонение', 'role': 'assistant', 'name': 'Пользователь'}, {'content': 'Вот как вы можете сгенерировать 1000 случайных чисел, рассчитать их среднее и стандартное отклонение в Python:\n\n```python\nimport numpy as np\n\n# Шаг 1: Сгенерируйте 1000 рандомных чисел\nrandom_numbers = np.random.rand(1000)\n\n# Шаг 2: Рассчитайте среднее и стандартное отклонение\nmean_value = np.mean(random_numbers)\nstd_deviation = np.std(random_numbers)\n\nprint("Среднее значение:", mean_value)\nprint("Стандартное отклонение:", std_deviation)\n```\n\nВ этом коде мы используем библиотеку NumPy для генерации случайных чисел и расчета статистических показателей. Функция `np.random.rand()` генерирует массив из 1000 случайных чисел в диапазоне от 0 до 1. Затем мы рассчитываем среднее значение с помощью функции `np.mean()` и стандартное отклонение с помощью функции `np.std()`

🔍 Что делает этот код
 1. UserProxyAgent — агент, имитирующий пользователя.

 2. AssistantAgent — агент, представляющий LLM, который будет отвечать.

 3. Инициация чата — пользователь задаёт вопрос, а ассистент пытается на него ответить.
 
 4. Автоматическая остановка происходит, если в ответе появится слово «конец».


### 📌 **CrewAI**
---
[Библиотека](https://www.crewai.com/)

**CrewAI** – фреймворк для построения команд автономных LLM-агентов с role-based организацией. 

#### Основные компоненты:
- **Crew (команда)** – верхнеуровневая организация, управляющая набором агентов и распределением задач
- **Agent (агент)** – специализированный участник с конкретной ролью, целью и инструментами
- **Task (задача)** – единичное задание с описанием и ожидаемым результатом
- **Process** – определяет схему выполнения задач (последовательную или иерархическую)
- **Flow** – дополнительный модуль для событийно-ориентированной оркестрации (Workflow) с возможностью условной логики и ветвлений

#### Агент может иметь менеджера (Manager Agent) при иерархическом процессе: в этом случае задаётся LLM и агент-менеджер для координации подчинённых 

In [29]:
# pip install crewai==0.30.7

In [30]:
# pip install crewai==0.134.0

In [28]:
# pip install crewai crewai-tools langchain langchain-community

In [10]:
from crewai import Agent, Task, Crew, LLM

ollama_llm = LLM(
  model="ollama/mistral-nemo:12b-instruct-2407-q3_K_L",
  api_base="http://localhost:11434",
  )

agent = Agent(role="Генератор идей", goal="Придумать стартапы", backstory="Ты визионер.", llm=ollama_llm, verbose=True)
task = Task(description="Придумай 5 оригинальных идей для технологических стартапов. Пиши на русском языке", 
            expected_output="Список из 5 идей.", agent=agent)
crew = Crew(agents=[agent], tasks=[task], verbose=True)
result = crew.kickoff()
print("\n🚀 Идеи:\n", result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d2c5d962-dc4e-4a30-9c3c-40d5a7932f12                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Генератор идей                                                                                          │
│                                                                                                                 │
│  Task: Придумай 5 оригинальных идей для технологических стартапов. Пиши на русском языке                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Генератор идей                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. **Агротех стартап - "Зеленая Планета"**                                                                     │
│     - Идея: Разработка и внедрение умных систем для сельского хозяйства, которые помогут фермерам               │
│  оптимизировать использование воды, удобрений, а также предотвращать болезни растений.                          │
│     - Технологии: IoT, Machine Learning, беспроводные датчики, блокчейн для отслеживания происхождения          │
│  продуктов.                                                                                                     │
│     - Рынок: Сельскохозяйственная промышленность.                                                               │
│                                                                                                                 │
│  2. **Стартап в области здравоохранения - "Здоровый Образ"**                                                    │
│     - Идея: Создание искусственного интеллекта, который поможет диагностировать заболевания на ранних стадиях   │
│  путем анализа изображений медицинских тестов и истории болезни пациента.                                       │
│     - Технологии: Deep Learning, Computer Vision, Natural Language Processing (NLP).                            │
│     - Рынок: Медицинская промышленность.                                                                        │
│                                                                                                                 │
│  3. **Стартап в области образования - "Умный Школьник"**                                                        │
│     - Идея: Разработка персонализированной платформы для обучения, которая использует Machine Learning для      │
│  адаптации содержимого и темпа обучения к индивидуальным потребностям учащегося.                                │
│     - Технологии: Machine Learning, Personalized Content Recommendation System.                                 │
│     - Рынок: Образовательные учреждения.                                                                        │
│                                                                                                                 │
│  4. **Стартап в области мобильных приложений - "Эко-карта"**                                                    │
│     - Идея: Создание геолокационного мобильного приложения, которое поможет пользователям находить              │
│  экологически чистые продукты и услуги поблизости.                                                              │
│     - Технологии: Геолокация, Mobile App Development.                                                           │
│     - Рынок: Забота об окружающей среде.                                                                        │
│                                                                                                                 │
│  5. **Стартап в области кибербезопасности - "Защитник Данных"**                                                 │
│     - Идея: Разработка решения для предотвращения и обнаружения кибератак на основе Machine Learning, которое   │
│  будет мониторить сетевой трафик и блокировать подозрительную активность в режиме реального времени.            │
│     - Технологии: Machine Learning, Network Traffic Monitoring, Intrusion Detection System (IDS).               │
│     - Рынок: Индустрия кибербезопасности.              

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a49d80de-13cb-431f-b02c-c4958843f7bc                                                                     │
│  Agent: Генератор идей                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d2c5d962-dc4e-4a30-9c3c-40d5a7932f12                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 1. **Агротех стартап - "Зеленая Планета"**                                                       │
│     - Идея: Разработка и внедрение умных систем для сельского хозяйства, которые помогут фермерам               │
│  оптимизировать использование воды, удобрений, а также предотвращать болезни растений.                          │
│     - Технологии: IoT, Machine Learning, беспроводные датчики, блокчейн для отслеживания происхождения          │
│  продуктов.                                                                                                     │
│     - Рынок: Сельскохозяйственная промышленность.                                                               │
│                                                                                                                 │
│  2. **Стартап в области здравоохранения - "Здоровый Образ"**                                                    │
│     - Идея: Создание искусственного интеллекта, который поможет диагностировать заболевания на ранних стадиях   │
│  путем анализа изображений медицинских тестов и истории болезни пациента.                                       │
│     - Технологии: Deep Learning, Computer Vision, Natural Language Processing (NLP).                            │
│     - Рынок: Медицинская промышленность.                                                                        │
│                                                                                                                 │
│  3. **Стартап в области образования - "Умный Школьник"**                                                        │
│     - Идея: Разработка персонализированной платформы для обучения, которая использует Machine Learning для      │
│  адаптации содержимого и темпа обучения к индивидуальным потребностям учащегося.                                │
│     - Технологии: Machine Learning, Personalized Content Recommendation System.                                 │
│     - Рынок: Образовательные учреждения.                                                                        │
│                                                                                                                 │
│  4. **Стартап в области мобильных приложений - "Эко-карта"**                                                    │
│     - Идея: Создание геолокационного мобильного приложения, которое поможет пользователям находить              │
│  экологически чистые продукты и услуги поблизости.                                                              │
│     - Технологии: Геолокация, Mobile App Development.                                                           │
│     - Рынок: Забота об окружающей среде.                                                                        │
│                                                                                                                 │
│  5. **Стартап в области кибербезопасности - "Защитник Данных"**                                                 │
│     - Идея: Разработка решения для предотвращения и обнаружения кибератак на основе Machine Learning, которое   │
│  будет мониторить сетевой трафик и блокировать подозрительную активность в режиме реального времени.            │
│     - Технологии: Machine Learning, Network Traffic Mo


🚀 Идеи:
 1. **Агротех стартап - "Зеленая Планета"**
   - Идея: Разработка и внедрение умных систем для сельского хозяйства, которые помогут фермерам оптимизировать использование воды, удобрений, а также предотвращать болезни растений.
   - Технологии: IoT, Machine Learning, беспроводные датчики, блокчейн для отслеживания происхождения продуктов.
   - Рынок: Сельскохозяйственная промышленность.

2. **Стартап в области здравоохранения - "Здоровый Образ"**
   - Идея: Создание искусственного интеллекта, который поможет диагностировать заболевания на ранних стадиях путем анализа изображений медицинских тестов и истории болезни пациента.
   - Технологии: Deep Learning, Computer Vision, Natural Language Processing (NLP).
   - Рынок: Медицинская промышленность.

3. **Стартап в области образования - "Умный Школьник"**
   - Идея: Разработка персонализированной платформы для обучения, которая использует Machine Learning для адаптации содержимого и темпа обучения к индивидуальным потребностям уча

### 📌 **SmolAgents**
---
[Библиотека](https://huggingface.co/docs/smolagents/index)


**SmolAgents** – минималистичная open-source-библиотека от Hugging Face для простого создания AI-агентов с помощью минимального количества кода. 

Главная идея – поддержать концепцию «код-агентов»: вместо генерации описания следующего действия в виде текста или JSON, агент сразу пишет и исполняет фрагменты Python-кода. Это позволяет максимально использовать возможности моделей при генерации кода и повышает гибкость логики агента.

При этом SmolAgents даёт полный доступ к множеству моделей и инструментов (поиск в интернете, базы знаний и пр.) и поддерживает разные режимы ввода (текст, изображение, аудио и др.) 


Ключевые возможности и сценарии:
- CodeAgent: основной класс агента, который выполняет Python-код. Такой подход снижает количество необходимых LLM-вызовов и улучшает композицию действий.

- Интеграция с инструментами: легко добавлять существующие «тулы» (например, веб-поиск, утилиты для работы с данными). Инструменты можно загружать и делиться через Hugging Face Hub.

- Поддержка моделей: можно использовать локальные трансформеры, или модели из разных провайдеров (OpenAI, Anthropic, HF), а также подключать модели через LiteLLM и другие обвязки.
- Примеры применения: быстрое прототипирование интерактивных агентов (ответы на запросы с последующим поиском или вычислениями), генерация SQL-запросов, ответы на вопросы с использованием веб-данных и т.д. 

In [ ]:
from huggingface_hub import login, InferenceClient

In [ ]:
from smolagents import CodeAgent, InferenceClientModel

model = InferenceClientModel() 

agent = CodeAgent(tools=[], model=model)

result = agent.run("Calculate the sum of numbers from 1 to 10")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Calculate the sum of numbers from 1 to 10                                                                       │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  total_sum = sum(range(1, 11))                                                                                    
  final_answer(total_sum)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 55

[Step 1: Duration 5.08 seconds| Input tokens: 2,008 | Output tokens: 63]

55


**Вывод**: 
    
    LangChain, AutoGen, CrewAI и SmolAgents представляют собой современные инструменты для построения интеллектуальных агентов на базе LLM. 
    
    LangChain – универсальный фреймворк с множеством компонентов для гибкого конструирования цепочек и агентов.
    AutoGen фокусируется на масштабируемой многопроцессной архитектуре с обширными возможностями мониторинга и распределения
    CrewAI предлагает подход «команда агентов» с ролевым распределением задач
    SmolAgents отличается компактностью и использованием «кодагентов» для максимальной простоты и эффективности .
    
    
На семинаре мы попробуем каждую и решим, где какой фреймворк будем использовать